# Assignment 1 - Ruoxin Shi 12334627

1. Downselect records corresponding to failed inspection (see "Results" column)
2. Clean the data, making sure that there are no NaNs in "Violations" column
3. "Violations" column lists the reasons for inspection failure. Those reasons are separated by "|".  Each reason consists of a regulation code, regulation description and comments describing how the regulation was violated. 
4. Using regular expression, parse "Violations" column to select only regulation descriptions
5. Count how many times each regulation description occurred in the table (hint: from collections import Counter)
6. Show top-10 the most frequent regulation descriptions
7. Your final output should be a Jupyter notebook showing all your code and the results so that one can easily reproduce them

Remember: you are presenting it to your boss and have very limited amount of time to state your case. Your presentation (charts and explanations within Jupyter notebook) should be as clear and short as possible but complete.

In [1]:
###Loading Packages###
import pandas as pd  # data frame operations  
import numpy as np  # arrays and math functions
import matplotlib.pyplot as plt  # static plotting
import seaborn as sns  # pretty plotting, including heat map
import re # regular expressions
import scipy
import os # Operation System
import sklearn
import math
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data
food = pd.read_csv('Food_Inspections.csv')

In [3]:
food

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2553618,HALF ACRE BEER COMPANY,HALF ACRE BEER COMPANY,2418217.0,Restaurant,Risk 1 (High),4253-4257 N LINCOLN AVE,CHICAGO,IL,60618.0,03/29/2022,Canvass,Out of Business,NaN,41.959233,-87.682153,"(-87.68215325256443, 41.95923309357485)"
1,2553580,POPEYE'S # 866,POPEYE'S,2738649.0,Restaurant,Risk 1 (High),7617 S RACINE AVE,CHICAGO,IL,60620.0,03/29/2022,Complaint Re-Inspection,Pass,NaN,41.755464,-87.653718,"(-87.65371781952274, 41.75546441273601)"
2,2553514,SUBWAY #24879,SUBWAY,2689652.0,Restaurant,Risk 1 (High),3951 N KIMBALL AVE,CHICAGO,IL,60618.0,03/25/2022,Non-Inspection,No Entry,NaN,41.953544,-87.712743,"(-87.71274346541044, 41.9535436902562)"
3,2553511,"ALL N'FAMILY SHRIMP SHACK, LLC","ALL N'FAMILY SHRIMP SHACK, LLC",2835227.0,Restaurant,Risk 2 (Medium),10341 S HALSTED ST,CHICAGO,IL,60628.0,03/25/2022,License,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.705698,-87.642650,"(-87.64265031197954, 41.70569754863335)"
4,2553459,EL TACONAZO EXPRESS INC,EL TACONAZO EXPRESS,2757771.0,Restaurant,Risk 1 (High),3129 W ARMITAGE AVE,CHICAGO,IL,60647.0,03/25/2022,Canvass Re-Inspection,Pass,NaN,41.917294,-87.706545,"(-87.70654528949485, 41.91729424455668)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233897,58246,THE JUICY WINE CO.,THE JUICY WINE CO.,1738131.0,Restaurant,Risk 2 (Medium),694 N MILWAUKEE AVE,CHICAGO,IL,60642.0,01/11/2010,Canvass,Pass w/ Conditions,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.894230,-87.652521,"(-87.65252070814081, 41.89422978422928)"
233898,58249,BAGEL ON DAMEN,BAGEL ON DAMEN,2009342.0,Restaurant,Risk 2 (Medium),1252 N DAMEN AVE,CHICAGO,IL,60622.0,01/12/2010,License,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.904884,-87.677451,"(-87.67745148114881, 41.90488420426394)"
233899,154218,ROBINSON'S RIBS,ROBINSON'S RIBS,45998.0,Restaurant,Risk 1 (High),655 W ARMITAGE AVE,CHICAGO,IL,60614.0,01/05/2010,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.918108,-87.645883,"(-87.6458831775194, 41.91810807444473)"
233900,154236,SUBWAY SANDWICHES AND SALADS,SUBWAY,2013762.0,Restaurant,Risk 2 (Medium),1513 W fullerton AVE,CHICAGO,IL,60614.0,01/14/2010,License Re-Inspection,Fail,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.925067,-87.666334,"(-87.66633415361996, 41.92506665893317)"


### 1. Downselect records corresponding to failed inspection (see "Results" column)

In [9]:
failed_df = food[food['Results'] == 'Fail']
failed_df

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
8,2553366,Qdoba Mexican Eats #2628,Qdoba Mexican Eats #2628,2017847.0,Restaurant,Risk 1 (High),957 W DIVERSEY PKWY,CHICAGO,IL,60614.0,03/23/2022,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.932531,-87.653709,"(-87.65370947965262, 41.93253117623138)"
20,1383429,WHITE PALACE GRILL,WHITE PALACE GRILL,1096426.0,Restaurant,Risk 1 (High),1159 S CANAL ST,CHICAGO,IL,60607.0,03/14/2014,Canvass,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.867319,-87.639057,"(-87.6390567460305, 41.86731940533911)"
28,2553082,HARRIS/KELLER CAFE,HARRIS/KELLER CAFE,2636450.0,Restaurant,Risk 1 (High),1307 E 60TH ST,CHICAGO,IL,60637.0,03/17/2022,Canvass,Fail,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,41.785958,-87.594310,"(-87.59430954359748, 41.78595783719046)"
30,2553081,SLICE FACTORY,SLICE FACTORY,2813369.0,Restaurant,Risk 2 (Medium),4422-4424 S PULASKI RD,CHICAGO,IL,60632.0,03/17/2022,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.812649,-87.723912,"(-87.7239121397429, 41.81264929452027)"
33,2553043,POPEYE'S #831,POPEYES,2594606.0,Restaurant,Risk 1 (High),1959 W HOWARD ST,CHICAGO,IL,60626.0,03/17/2022,Complaint Re-Inspection,Fail,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",42.019333,-87.680022,"(-87.68002153080553, 42.0193331123976)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233877,158271,CHINA WOK,CHINA WOK,2013324.0,Restaurant,Risk 1 (High),1958 W 35TH ST,CHICAGO,IL,60609.0,01/19/2010,License Re-Inspection,Fail,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.830500,-87.675221,"(-87.67522140471041, 41.83050041795062)"
233883,67924,FRANK FOOD #4,FRANK FOOD #4,1767467.0,Grocery Store,Risk 3 (Low),1819 S KEDZIE AVE,CHICAGO,IL,60623.0,01/19/2010,Complaint,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.857141,-87.705261,"(-87.70526116851221, 41.85714095488576)"
233895,164258,LUMES PANCAKE HOUSE,LUMES PANCAKE HOUSE,39623.0,Restaurant,Risk 1 (High),11601 S WESTERN AVE,CHICAGO,IL,60643.0,01/07/2010,Complaint,Fail,29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42...,41.682672,-87.680726,"(-87.6807260850454, 41.682671696042505)"
233898,58249,BAGEL ON DAMEN,BAGEL ON DAMEN,2009342.0,Restaurant,Risk 2 (Medium),1252 N DAMEN AVE,CHICAGO,IL,60622.0,01/12/2010,License,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.904884,-87.677451,"(-87.67745148114881, 41.90488420426394)"


### 2. Clean the data, making sure that there are no NaNs in "Violations" column

In [14]:
failed_df = failed_df.dropna(subset=['Violations']).reset_index()

###  3. Parse "Violations" column to select only regulation descriptions

**Parse one violation - find the pattern**

In [15]:
failed_df['Violations'][0]

"10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: WE FOUND HOT WATER TEMPERATURE AT 68.7F TO 75.9F IN THE WASHBOWL IN THE MEN'S WASHROOM. MUST PROVIDE HOT WATER WITH A MINIMUM TEMPERATURE OF 100F AT SAID WASHBOWL. ADEQUATE HOT WATER FOUND IN WOMEN'S WASHROOM.  PRIORITY VIOLATION 7-38-030(C),CITATION ISSUED | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: EXCESS WATER ON FLOOR UNDER THREE COMPARTMENT SINK. CLEAN MAINTAIN AND KEEP AREA DRY."

In [30]:
# Term to split on
split_term = '\|'
# split the reasons
failed_df['splitted_reasons'] = failed_df['Violations'].apply(lambda x: re.split(split_term, x))
failed_df['descriptions'] = None

In [29]:
failed_df.iloc[0]['splitted_reasons']

["10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: WE FOUND HOT WATER TEMPERATURE AT 68.7F TO 75.9F IN THE WASHBOWL IN THE MEN'S WASHROOM. MUST PROVIDE HOT WATER WITH A MINIMUM TEMPERATURE OF 100F AT SAID WASHBOWL. ADEQUATE HOT WATER FOUND IN WOMEN'S WASHROOM.  PRIORITY VIOLATION 7-38-030(C),CITATION ISSUED ",
 ' 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: EXCESS WATER ON FLOOR UNDER THREE COMPARTMENT SINK. CLEAN MAINTAIN AND KEEP AREA DRY.']

In [20]:
# Term to split on
split_term = '\|'

# Split the phrase
reasons = re.split(split_term, failed_df['Violations'][0])

In [23]:
# find patterns
comment = "\s\-\sComments:[\s\S]*"
regulation_code = "[\0-9]+\.\s"

# delete comment
no_comment = re.sub(comment, '', reasons[0])

# delete regulation code
re.sub(regulation_code, '', no_comment)

'ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE'

**Pasrse all violations**

In [68]:
# find patterns
comment = "\s\-\sComments:[\s\S]*"
regulation_code = "[\0-9]+\.\s"

result = []

for i in range(failed_df.shape[0]):
    reasons = failed_df.iloc[i]['splitted_reasons']
    
    description = []
    for r in reasons:
        # delete comment
        no_comment = re.sub(comment, '', r)

        # delete regulation code
        no_code = re.sub(regulation_code, '', no_comment)
        no_code = no_code.strip()
        description.append(no_code)
    
#     print("--------Row:", i)
#     print(description)
    result.append(description)

In [69]:
failed_df['descriptions'] = result

### 4. Count how many times each regulation description occurred in the table (hint: from collections import Counter)

In [77]:
from collections import Counter

flat_result = [item for l in result for item in l]
c = Counter(flat_result)
#print(c)

In [72]:
cnt = pd.DataFrame.from_dict(c, orient='index').reset_index()
cnt = cnt.rename(columns={'index':'Description', 0:'Frequency'})


### 5. Show top-10 the most frequent regulation descriptions

In [83]:
top10 = cnt.sort_values(by="Frequency", ascending=False).reset_index().head(10).drop(columns=["index"])

**Below are the top10 most frequent regulation descriptions with frequencies**

In [84]:
top10

,Description,Frequency
0,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED",19371
1,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS",18257
2,"FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS",16445
3,"NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS",16432
4,VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED,15529
5,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED",15019
6,"PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN",12570
7,"PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING EQUIPMENT PROPERLY STORED",10667
8,"LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED",7736
9,"INSECTS, RODENTS, & ANIMALS NOT PRESENT",6582


In [40]:
print(c.most_common(10))

[('FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED', 19344), ('WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS', 18227), ('FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS', 16425), ('NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS', 16421), ('VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED', 15515), ('FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED', 14995), ('PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN', 12568), ('PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED', 10662), ('LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED', 7724), ('INSECTS, RODENTS, & ANIMALS NOT PRESENT', 6578)]


In [64]:
pd.set_option('display.max_colwidth', None)